In [9]:
'''
Considerations
-train neural net within ages of 2-6 Gyrs: we know M67 has an age of about 4Gyrs, so there is no point to training it outside this range
-We want to have a working neural net that works to a precision of 0.005 dex (0.5% precision), though we should make a determination of how accurate we want it to be. 
-Most logical loss function is MAE (mean absolute error), although the Median Absolute Error is smaller. We probably want to train using the Mean Absolute Error and then evaluate using the Median Absolute Error.
-To determine an adequate guess than the number of epochs to train for we can take a small portion of the grid (e.g. say only where the metallicity is solar and take a few masses) train the neural net till it reaches the desired precision, suggested architecture (3,8,8,2) = (inputs,nodes, hidden layers, outputs).
-Sensible number of epochs = 50,000
-We want to try an architecture which is the smallest we can get away with such that its faster (start with small architecture and work up).
'''
import numpy as np
data = np.genfromtxt('grid_0_0.csv',delimiter=",",skip_header=1)

'''
[0]=row
[1]=model_number
[2]=star_mass
[3]=star_age
[4]=star_feh
[5]=star_MLT
[6]=star_fov
[7]=effective_T
[8]=luminosity
[9]=radius
[10]=log_g
[11]=surface_h1
[12]=surface_he4
[13]=center_h1
[14]=center_he4
[15]=log_center_T
[16]=log_center_Rho
[17]=he_core_mass
[18]=delta_nu
[19]=delta_Pg
[20]=nu_max
[21]=nu_0_10
[22]=nu_0_11
[23]=nu_0_12
[24]=nu_0_13
[25]=nu_0_14
[26]=nu_0_15
'''

'\n[0]=row\n[1]=model_number\n[2]=star_mass\n[3]=star_age\n[4]=star_feh\n[5]=star_MLT\n[6]=star_fov\n[7]=effective_T\n[8]=luminosity\n[9]=radius\n[10]=log_g\n[11]=surface_h1\n[12]=surface_he4\n[13]=center_h1\n[14]=center_he4\n[15]=log_center_T\n[16]=log_center_Rho\n[17]=he_core_mass\n[18]=delta_nu\n[19]=delta_Pg\n[20]=nu_max\n[21]=nu_0_10\n[22]=nu_0_11\n[23]=nu_0_12\n[24]=nu_0_13\n[25]=nu_0_14\n[26]=nu_0_15\n'

In [70]:
'''
def find_tracks(ages):
    tracks = []
    current_age_val = ages[0]
    track_start_index = 0
    for i in range(len(ages)):
        if ages[i] < current_age_val:
            print([i,ages[i],current_age_val])
            tracks.append([track_start_index,i-1])
            track_start_index = i
        current_age_val = ages[i]
    tracks.append([track_start_index,len(ages)-1])
    return tracks
tracks = find_tracks(ages=data[:,3])
'''
def find_tracks(masses):
    tracks = []
    current_mass_val = masses[0]
    track_start_index = 0
    for i in range(len(masses)):
        if masses[i] != current_mass_val:
            #print([i,masses[i],current_mass_val])
            tracks.append([track_start_index,i-1])
            track_start_index = i
        current_mass_val = masses[i]
    tracks.append([track_start_index,len(masses)-1])
    return tracks
#tracks = find_tracks(masses=data[:,2])
def limit_age_range(lower_bounds,upper_bounds,data):
    tracks = find_tracks(masses=data[:,2])
    limited_tracks = []
    for i in range(len(tracks)):
        track = data[:,3][tracks[i][0]:tracks[i][1]+1]
        lb = (np.abs(track - lower_bounds)).argmin() + tracks[i][0]
        ub = (np.abs(track - upper_bounds)).argmin()+ tracks[i][0]
        if data[:,3][ub] > lower_bounds: #makes stars that are really young aren't included if their maximum age is less than the lower bounds
            limited_tracks.append([lb,ub])
    
    return limited_tracks
tracks = limit_age_range(lower_bounds=2,upper_bounds=6,data=data)

for i in range(len(tracks)):
    track = data[:,3][tracks[i][0]:tracks[i][1]+1]
    print([track[0],track[-1]])
    #print([data[:,3][tracks[i][0]],data[:,3][tracks[i][1]]])
    #print([data[:,3][tracks[i][0]+1],data[:,3][tracks[i][1]+1]])

[2.0137011296318104, 6.0063306108872725]
[2.047949197098893, 6.040578678354355]
[1.982241666191012, 5.9877245396828895]
[2.010917625158262, 6.005389056671119]
[1.976569299994252, 5.759212269577462]
[1.976096966924976, 4.309989670279712]
[1.9939831832258756, 3.3130918417258326]
[2.0017896988809047, 2.6175492312392463]
[2.001309665882801, 2.127369869516874]
